# Utilizing PPO RL agent as trainable and optimizing the hyperparameters in the Cartpole environment

First we need to define the search space for the PPO hyperparameters we want to tune/optimize. So, we can check the [PPO config training](https://docs.ray.io/en/latest/rllib/rllib-algorithms.html#ray.rllib.algorithms.ppo.ppo.PPOConfig) in the Ray RLlib page to verify the specific PPO hyperparameters, and also the [general RL algorithm hyperparameters](https://docs.ray.io/en/latest/rllib/package_ref/doc/ray.rllib.algorithms.algorithm_config.AlgorithmConfig.training.html#ray-rllib-algorithms-algorithm-config-algorithmconfig-training).

For this notebook, we will try to optimize the learning rate `lr` and the discount factor `gamma` hyperparameters. We can check the common values assumed for these variables using the SB3 Zoo [here](https://github.com/DLR-RM/rl-baselines3-zoo/blob/master/rl_zoo3/hyperparams_opt.py). These values were defined based on community experience using the software and algorithms.

In [1]:
from pathlib import Path
from ray import air, tune
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.algorithm import Algorithm

In [2]:
search_space = {
    "lr": tune.loguniform(1e-5, 1),
    "gamma": tune.choice(
        [
            0.5,
            0.6,
            0.7,
            0.8,
            0.9,
            0.95,
            0.98,
            0.99,
            0.995,
            0.999,
            0.9999,
        ]
    ),
}

You can check the `loguniform` and the `choice` search space [here](https://docs.ray.io/en/latest/tune/api/search_space.html).

In Ray Tune, there is a Search Space algorithm responsible for selecting the hyperparameters to be used in a trial. In this example, we are using a Random Search algorithm that randomly selects samples from the hyperparameters in the search space.

In [3]:
search_algo = tune.search.basic_variant.BasicVariantGenerator()  # Random search

In Ray Tune, there is also a Trial Scheduler algorithm responsible for early terminate bad trials, pause trials, clone trials, and alter hyperparameters of a running trial. In this example, we are considering a simple scheduler that only allocates the trials in the FIFO order without pausing, terminating or altering hyperparameters of a running trial.

In [4]:
scheduler_algo = tune.schedulers.FIFOScheduler()  # FIFO trial scheduler

Once the search and scheduler algorithms are defined, we can define our Tune configuration:

In [5]:
number_trials = 2
tune_config = tune.TuneConfig(
    metric="env_runners/episode_reward_mean",  # That's the metric we want to maximize/minimize
    mode="max",  # Here we indicate we want to maximize the metric env_runners/episode_reward_mean
    scheduler=scheduler_algo,
    search_alg=search_algo,
    num_samples=number_trials,  # Number of trials to run
)

Now, let's initialize our Tuner and train the PPO RL agent.

In [6]:
config = PPOConfig().environment("CartPole-v1")
stop = {
    "training_iteration": 10,
}
checkpoint_frequency = 0
store_results_path = str(Path("./ray_results/").resolve()) + "/nb_2/"
agent_name = "ppo_cartpole"

tuner = tune.Tuner(
    "PPO",
    param_space={
        **config.to_dict(),
        **search_space,
    },  # Here we mix the Algo config with the search space
    tune_config=tune_config,
    run_config=air.RunConfig(
        storage_path=store_results_path,
        name=agent_name,
        stop=stop,
        verbose=2,
        checkpoint_config=air.CheckpointConfig(
            checkpoint_frequency=checkpoint_frequency,
            checkpoint_at_end=True,
        ),
    ),
)
results = tuner.fit()
print(results)

2024-11-30 02:50:09,520	INFO worker.py:1783 -- Started a local Ray instance.
2024-11-30 02:50:10,043	INFO tune.py:253 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `Tuner(...)`.
2024-11-30 02:50:10,045	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/lasse/.local/share/virtualenvs/ray_minicourse-PTDOXG61/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, act

(PPO pid=515765) Install gputil for GPU system monitoring.
(PPO pid=515765) 2024-11-30 02:50:19,838	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!


Trial name,agent_timesteps_total,counters,custom_metrics,env_runners,episode_media,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_CartPole-v1_cfa8c_00000,40000,"{'num_env_steps_sampled': 40000, 'num_env_steps_trained': 40000, 'num_agent_steps_sampled': 40000, 'num_agent_steps_trained': 40000}",{},"{'episode_reward_max': 500.0, 'episode_reward_min': 30.0, 'episode_reward_mean': np.float64(278.86), 'episode_len_mean': np.float64(278.86), 'episode_media': {}, 'episodes_timesteps_total': 27886, 'policy_reward_min': {'default_policy': np.float64(30.0)}, 'policy_reward_max': {'default_policy': np.float64(500.0)}, 'policy_reward_mean': {'default_policy': np.float64(278.86)}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [79.0, 59.0, 30.0, 99.0, 226.0, 171.0, 131.0, 256.0, 184.0, 77.0, 152.0, 180.0, 94.0, 136.0, 54.0, 227.0, 232.0, 167.0, 85.0, 153.0, 195.0, 162.0, 167.0, 120.0, 172.0, 295.0, 154.0, 203.0, 324.0, 91.0, 250.0, 57.0, 64.0, 103.0, 286.0, 275.0, 233.0, 166.0, 139.0, 267.0, 340.0, 178.0, 237.0, 99.0, 357.0, 206.0, 66.0, 383.0, 389.0, 288.0, 261.0, 388.0, 322.0, 202.0, 332.0, 310.0, 188.0, 348.0, 221.0, 86.0, 99.0, 291.0, 302.0, 500.0, 369.0, 492.0, 367.0, 249.0, 363.0, 279.0, 203.0, 324.0, 122.0, 500.0, 500.0, 324.0, 296.0, 460.0, 500.0, 500.0, 359.0, 418.0, 348.0, 294.0, 500.0, 468.0, 500.0, 412.0, 500.0, 500.0, 500.0, 500.0, 500.0, 427.0, 500.0, 500.0, 500.0, 500.0, 404.0, 500.0], 'episode_lengths': [79, 59, 30, 99, 226, 171, 131, 256, 184, 77, 152, 180, 94, 136, 54, 227, 232, 167, 85, 153, 195, 162, 167, 120, 172, 295, 154, 203, 324, 91, 250, 57, 64, 103, 286, 275, 233, 166, 139, 267, 340, 178, 237, 99, 357, 206, 66, 383, 389, 288, 261, 388, 322, 202, 332, 310, 188, 348, 221, 86, 99, 291, 302, 500, 369, 492, 367, 249, 363, 279, 203, 324, 122, 500, 500, 324, 296, 460, 500, 500, 359, 418, 348, 294, 500, 468, 500, 412, 500, 500, 500, 500, 500, 427, 500, 500, 500, 500, 404, 500], 'policy_default_policy_reward': [79.0, 59.0, 30.0, 99.0, 226.0, 171.0, 131.0, 256.0, 184.0, 77.0, 152.0, 180.0, 94.0, 136.0, 54.0, 227.0, 232.0, 167.0, 85.0, 153.0, 195.0, 162.0, 167.0, 120.0, 172.0, 295.0, 154.0, 203.0, 324.0, 91.0, 250.0, 57.0, 64.0, 103.0, 286.0, 275.0, 233.0, 166.0, 139.0, 267.0, 340.0, 178.0, 237.0, 99.0, 357.0, 206.0, 66.0, 383.0, 389.0, 288.0, 261.0, 388.0, 322.0, 202.0, 332.0, 310.0, 188.0, 348.0, 221.0, 86.0, 99.0, 291.0, 302.0, 500.0, 369.0, 492.0, 367.0, 249.0, 363.0, 279.0, 203.0, 324.0, 122.0, 500.0, 500.0, 324.0, 296.0, 460.0, 500.0, 500.0, 359.0, 418.0, 348.0, 294.0, 500.0, 468.0, 500.0, 412.0, 500.0, 500.0, 500.0, 500.0, 500.0, 427.0, 500.0, 500.0, 500.0, 500.0, 404.0, 500.0]}, 'sampler_perf': {'mean_raw_obs_processing_ms': np.float64(0.19439578188943432), 'mean_inference_ms': np.float64(0.6707228199427331), 'mean_action_processing_ms': np.float64(0.07895767599599109), 'mean_env_wait_ms': np.float64(0.04029074512553687), 'mean_env_render_ms': np.float64(0.0)}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': np.float64(0.0043277740478515625), 'StateBufferConnector_ms': np.float64(0.003129243850708008), 'ViewRequirementAgentConnector_ms': np.float64(0.08161735534667969)}, 'num_episodes': 8, 'episode_return_max': 500.0, 'episode_return_min': 30.0, 'episode_return_mean': np.float64(278.86), 'episodes_this_iter': 8}",{},"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(0.36715484), 'cur_kl_coeff': np.float64(0.01875), 'cur_lr': np.float64(1.3985265446433982e-05), 'total_loss': np.float64(9.929167386536957), 'policy_loss': np.float64(-0.0173306

(PPO pid=515764) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/lasse/ray_minicourse/lesson_2/ray_results/nb_2/ppo_cartpole/PPO_CartPole-v1_cfa8c_00000_0_gamma=0.9950,lr=0.0000_2024-11-30_02-50-10/checkpoint_000000)
(PPO pid=515764) Install gputil for GPU system monitoring.
(PPO pid=515764) 2024-11-30 02:50:19,875	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!
2024-11-30 02:51:52,274	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/lasse/ray_minicourse/lesson_2/ray_results/nb_2/ppo_cartpole' in 0.0166s.
2024-11-30 02:51:52,763	INFO tune.py:1041 -- Total run time: 102.72 seconds (102.18 seconds for the tuning loop).


ResultGrid<[
  Result(
    metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': np.float64(0.0), 'grad_gnorm': np.float32(0.36715484), 'cur_kl_coeff': np.float64(0.01875), 'cur_lr': np.float64(1.3985265446433982e-05), 'total_loss': np.float64(9.929167386536957), 'policy_loss': np.float64(-0.01733068605904938), 'vf_loss': np.float64(9.94645794078868), 'vf_explained_var': np.float64(-0.06159804110885948), 'kl': np.float64(0.0021413145434349998), 'entropy': np.float64(0.5399863363594137), 'entropy_coeff': np.float64(0.0)}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': np.float64(128.0), 'num_grad_updates_lifetime': np.float64(8835.5), 'diff_num_grad_updates_vs_sampler_policy': np.float64(464.5)}}, 'num_env_steps_sampled': 40000, 'num_env_steps_trained': 40000, 'num_agent_steps_sampled': 40000, 'num_agent_steps_trained': 40000}, 'env_runners': {'episode_reward_max': 500.0, 'episode_reward_min': 30

In the logs above, you can check that there are two different trials utilizing different learning rate `lr` and discount factor `gamma` values. Confirming that they sampled different configurations based on the defined search space.

Finally, you can check the different training reward performances using tensorboard and looking for the metric `episode_reward_mean`. You can observe two different curves (one for each trial) showing the reward performance over the training.

In [8]:
%load_ext tensorboard
%tensorboard --logdir ray_results/nb_2

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 516186), started 0:00:11 ago. (Use '!kill 516186' to kill it.)